In [5]:
import re
from underthesea import word_tokenize
from rank_bm25 import BM25Okapi, BM25Plus
import pickle

In [6]:
file = open('data.txt',encoding='utf-8').read().split("\n")
data = []
for line in file:
    if re.search("^.*\.", line) or re.search("^.*\)", line):
        data.append(line.lower())

stopwords = open('../stopwords.txt', encoding='utf-8').read().split(" ")

In [7]:
def removeStopwords(sentence):
  words = word_tokenize(sentence)
  word = [w.lower() for w in words if w not in (stopwords)]
  sentence_clean = " ".join(word)
  
  return sentence_clean

In [8]:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    top_results_100words = [' '.join(top_result.split(' ')) 
                             for top_result in top_results]
    
    return top_results_100words

In [9]:
tokenized_data = [word_tokenize(doc) for doc in data]


In [11]:
bm25 = BM25Okapi(tokenized_data)

In [26]:
# Save model to load
with open('bm25result', 'wb') as bm25result_file:
    pickle.dump(bm25, bm25result_file)

In [27]:
#to read bm25 object
with open('bm25result', 'rb') as bm25result_file:
    bm25result = pickle.load(bm25result_file)

In [28]:
query = "bộ luật dân sự là gì ?"
print(removeStopwords(query))
tokenized_query = word_tokenize(removeStopwords(query))

bm25okapi_search(tokenized_query = tokenized_query,
                 bm25 = bm25, 
                 corpus = data,
                 n_results = 5)

bộ luật dân sự


['đối với giao dịch dân sự được xác lập trước ngày bộ luật này có hiệu lực thì việc áp dụng pháp luật được quy định như sau: giao dịch dân sự chưa được thực hiện mà có nội dung, hình thức khác với quy định của bộ luật này thì chủ thể giao dịch tiếp tục thực hiện theo quy định của bộ luật dân sự số 33/2005/qh11 và các văn bản quy phạm pháp luật quy định chi tiết bộ luật dân sự số 33/2005/qh11, trừ trường hợp các bên của giao dịch dân sự có thỏa thuận về việc sửa đổi, bổ sung nội dung, hình thức của giao dịch để phù hợp với bộ luật này và để áp dụng quy định của bộ luật này, giao dịch dân sự đang được thực hiện mà có nội dung, hình thức khác với quy định của bộ luật này thì áp dụng quy định của bộ luật dân sự số 33/2005/qh11 và các văn bản quy phạm pháp luật quy định chi tiết bộ luật dân sự số 33/2005/qh11, giao dịch dân sự chưa được thực hiện hoặc đang được thực hiện mà có nội dung và hình thức phù hợp với quy định của bộ luật này thì áp dụng quy định của bộ luật này, giao dịch dân sự đ

In [31]:
query = "Hiệu lực thi hành bộ luật dân sự là khi nào ?"
print("input question: ", query)
print("Cleaned question: ",removeStopwords(query))
tokenized_query = word_tokenize(removeStopwords(query))
bm25okapi_search(tokenized_query = tokenized_query,
                 bm25 = bm25, 
                 corpus = data,
                 n_results = 1)

input question:  Hiệu lực thi hành bộ luật dân sự là khi nào ?
Cleaned question:  hiệu lực thi hành bộ luật dân sự nào


['bộ luật này có hiệu lực thi hành từ ngày 01 tháng 01 năm 2017, bộ luật dân sự số 33/2005/qh11 hết hiệu lực kể từ ngày bộ luật này có hiệu lực, bộ luật này đã được quốc hội nước cộng hòa xã hội chủ nghĩa việt nam khóa xiii, kỳ họp thứ 10 thông qua ngày 24 tháng 11 năm 2015 ; điều 689.']

In [1]:
from datasets import Dataset
import json

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function read file test
def read_data_test(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    question = []
    answer = []
    for object in fileRead:
        question.append(object["question"].lower())
        answer.append(object["answer"].lower())
    dataset = {
    "question": question,
    "answer": answer
    }
    return dataset

In [3]:
# To get score predict with bleu
test_data = read_data_test("./QA_data/qa_test.json")
test_data = Dataset.from_dict(test_data)

In [13]:
predicts = []
for i in test_data["question"]:
    tokenized_query = word_tokenize(removeStopwords(i))
    result = bm25okapi_search(tokenized_query = tokenized_query,
                 bm25 = bm25, 
                 corpus = data,
                 n_results = 1)
    predicts.append(result)
    

In [18]:
from nltk.translate.bleu_score import sentence_bleu
#function to get score of results
def getScore(ref, candi):
    score = sentence_bleu(ref, candi)
    return float('{:.4f}'.format(score))

In [20]:
reference= []
for i in test_data["answer"]:
    reference.append(i.split())


In [35]:
scores = []
for i in predicts:
    scores.append(getScore(reference,i[0].split()))

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [36]:
average = sum(scores)/len(scores)
average

0.42416091954023005

In [37]:
import datasets
rouge = datasets.load_metric("rouge")
rouge_output = rouge.compute(predictions=predicts, references=test_data["answer"], rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)


C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


Score(precision=0.265196193452529, recall=0.8852439869308493, fmeasure=0.3662911535955582)
